In [88]:
import findspark
findspark.init()

import pyspark
import pandas as pd 
import numpy as np
import os 
import datetime
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as mticker
import plotly.express as px

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql.window import Window
from math import radians, cos, sin, asin, sqrt
from pyspark.sql import functions as F
from pyspark.sql.functions import col, row_number, count, when, isnan, abs, cos, atan2

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from cmath import pi
from math import atan2, radians

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [89]:
gdrive_path = 'C:\\Users\\matts\\Google Drive\\berkeley\\.shortcut-targets-by-id\\11wLy1WKwOTcthBs1rpfEzkqax2BZG-6E\\W210_Capstone\\Data\\schools\\joined_datetime_with_income_10-5-22.parquet'

In [90]:
df_all = pd.read_parquet(gdrive_path)

In [11]:
display(df_all)

,year,cdscode,school_county,school_zip,school_region_name,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability
0,2010.0,20102070109728,Madera,93610,Northern San Joaquin Valley,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0
1,2010.0,20102072030187,Madera,93610,Northern San Joaquin Valley,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0
2,2010.0,20651930000000,Madera,93610,Northern San Joaquin Valley,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0
3,2010.0,20651930113076,Madera,93610,Northern San Joaquin Valley,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0
4,2010.0,20651936023907,Madera,93610,Northern San Joaquin Valley,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,Superior California,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.838727,32079603.0
2983107,2016.0,31669446031298,Placer,96161,Superior California,1980-0

In [91]:
df_all = df_all.drop(columns=['school_region_name'])

In [182]:
display(df_all)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.838727,32079603.0
2983107,2016.0,31669446031298,Placer,96161,1980-07-01,NaT,39.316133,-120.455805,0.168952,1840.70,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,

In [183]:
county_lookup_v2 = pd.read_csv('C:\\Users\\matts\\Documents\\Berkeley MIDS\\DataSci 210 Capstone\\non-push files\\county_zip_lookup.csv')

In [184]:
display(county_lookup_v2)

,school_zip,school_county_v2
0,90001,Los Angeles
1,90002,Los Angeles
2,90003,Los Angeles
3,90004,Los Angeles
4,90005,Los Angeles
...,...,...
2591,96361,Placer
2592,89447,NaN
2593,91719,Riverside
2594,91742,Los Angeles


In [21]:
df_all.dtypes

year                                float64
cdscode                              object
school_county                        object
school_zip                           object
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male                object
population_10_14_female         

In [94]:
df_all = df_all.astype({'school_zip':'int32'})

In [80]:
df_all.dtypes

year                                float64
cdscode                              object
school_county                        object
school_zip                            int32
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male                object
population_10_14_female         

In [20]:
county_lookup_v2.dtypes

school_zip     int64
county_v2     object
dtype: object

In [185]:
df_all_augment = pd.merge(df_all, county_lookup_v2, on="school_zip", how="left")

In [186]:
display(df_all_augment)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.838727,32079603.0,Nevada
2983107,2016.0,31669446031298,Placer,96161,1980-07-01,NaT,39.316133,-120.455805,0.168952,1840.70,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,

In [187]:
df_regions = pd.read_csv('C:\\Users\\matts\\Google Drive\\berkeley\\.shortcut-targets-by-id\\11wLy1WKwOTcthBs1rpfEzkqax2BZG-6E\\W210_Capstone\\Data\\county region lookup.csv')

In [188]:
df_regions = df_regions.drop(columns=['school_region_number'])

In [189]:
display(df_regions)

,school_region_name,school_county_v2
0,Superior California,Butte
1,Superior California,Colusa
2,Superior California,El Dorado
3,Superior California,Glenn
4,Superior California,Lassen
5,Superior California,Modoc
6,Superior California,Nevada
7,Superior California,Placer
8,Superior California,Plumas
9,Superior California,Sacramento


In [190]:
df_merged = pd.merge(df_all_augment, df_regions, on='school_county_v2', how='left')

In [121]:
display(df_merged)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.838727,32079603.0,Nevada,Superior California
298310

In [191]:
null_rows = df_merged[['school_lat', 'school_lon','school_zip','school_county_v2']][df_merged[['school_zip','school_county_v2']].isnull().any(axis=1)]

In [150]:
display(null_rows)

,school_lat,school_lon,school_zip,school_county_v2
593128,39.0824,-119.2184,89447,NaN


In [161]:
df_nearby = pd.read_csv('C:\\Users\\matts\\Google Drive\\berkeley\\.shortcut-targets-by-id\\11wLy1WKwOTcthBs1rpfEzkqax2BZG-6E\\W210_Capstone\\Data\\schools\\pointsources_within_2250m_by_school_by_year_lookup.csv')

In [162]:
display(df_nearby)

,cdscode,lookup_year,nearby_ps_count
0,1611190134304,2014,1
1,1611196090054,2002,2
2,1611760127233,2014,1
3,1612346001481,2008,2
4,1612590110189,2005,2
...,...,...,...
22532,50755566052971,2008,2
22533,50757396053144,2017,1
22534,52105200000000,2005,1
22535,54755316107650,2014,2


In [147]:
df_nearby.dtypes

cdscode            float64
lookup_year          int64
nearby_ps_count      int64
dtype: object

In [192]:
df_merged = df_merged.astype({'cdscode':'int64'})

In [152]:
display(df_merged)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.838727,32079603.0,Nevada,Superior California
298310

In [153]:
year_lookup = pd.read_csv('C:\\Users\\matts\\Google Drive\\berkeley\\.shortcut-targets-by-id\\11wLy1WKwOTcthBs1rpfEzkqax2BZG-6E\\W210_Capstone\\Data\\school_pollution_mapping\\year_lookup.csv')

In [154]:
display(year_lookup)

,year,lookup_year
0,2000,2002
1,2001,2002
2,2002,2002
3,2003,2005
4,2004,2005
5,2005,2005
6,2006,2008
7,2007,2008
8,2008,2008
9,2009,2011


In [158]:
year_lookup.dtypes

year           int64
lookup_year    int64
dtype: object

In [193]:
df_merged = df_merged.merge(year_lookup, how='left', on=['year'])

In [169]:
display(df_merged)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,lookup_year
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.

In [157]:
df_merged.dtypes

year                                float64
cdscode                               int64
school_county                        object
school_zip                            int32
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male                object
population_10_14_female         

In [194]:
df_merged = df_merged.merge(df_nearby, how='left', on=['cdscode', 'lookup_year'])

In [195]:
null_rows = df_merged[['year','cdscode','lookup_year','nearby_ps_count']][df_merged[['year','cdscode','lookup_year','nearby_ps_count']].isnull().any(axis=1)]
display(null_rows)

,year,cdscode,lookup_year,nearby_ps_count
2,2010.0,20651930000000,2011.0,NaN
3,2010.0,20651930113076,2011.0,NaN
8,2010.0,15636770113837,2011.0,NaN
9,2010.0,15636770114512,2011.0,NaN
10,2010.0,15636776009823,2011.0,NaN
...,...,...,...,...
2983106,2016.0,31669443138500,2017.0,NaN
2983107,2016.0,31669446031298,2017.0,NaN
2983108,2016.0,31669446031322,2017.0,NaN
2983109,2016.0,31668520120105,2017.0,NaN


In [196]:
display(df_merged)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,lookup_year,nearby_ps_count
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,1.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,1.0
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,NaN
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,NaN
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1

In [197]:
df_merged['zero'] = 0

In [198]:
df_merged['nearby_point_source_count'] = df_merged.nearby_ps_count.combine_first(df_merged.zero)

In [199]:
display(df_merged)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,lookup_year,nearby_ps_count,zero,nearby_point_source_count
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,1.0,0,1.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,1.0,0,1.0
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,NaN,0,0.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,NaN,0,0.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,2011.0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.42

In [200]:
df_merged = df_merged.drop(columns=['zero','nearby_ps_count','lookup_year'])

In [201]:
display(df_merged)

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.000,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983106,2016.0,31669443138500,Placer,96161,1980-07-01,NaT,39.324762,-120.214355,0.168952,1794.66,2016-06,18255.0,9580.0,8675.0,1113.555,450.26,667.975,1259.595,766.4,494.475,1405.635,670.6,737.375,967.515,622.7,355.675,2509.960,2255.5,4746.3,1.9043859,39.32004,-120.13956,1797.86,1.693825957653,6470.985860231626,-85.32941655311153,39.184,-120.427,0.168952,2.570111,2.570111,87.899527,87.899527,0.168952,87.899527,1.036652,2016-01-01,2,2016.0,96161,96568.8

In [ ]:
df_merged.to_parquet('C:\\Users\\matts\\Google Drive\\berkeley\\.shortcut-targets-by-id\\11wLy1WKwOTcthBs1rpfEzkqax2BZG-6E\\W210_Capstone\\Data\\joined_data\\joined_data_fixed_counties_10-6-22.parquet')

In [180]:
null_rows = df_merged[['school_lat', 'school_lon','school_zip','school_county_v2', 'school_region_name']][df_merged[['school_lat', 'school_lon','school_zip','school_county_v2', 'school_region_name']].isnull().any(axis=1)]

In [181]:
display(null_rows)

,school_lat,school_lon,school_zip,school_county_v2,school_region_name
593128,39.0824,-119.2184,89447,NaN,Superior California
